INSTALL DEPENDENCIES

In [ ]:
!pip install pandas
!pip install scikit-learn
!pip install numpy
!pip install onnxmltools
!pip install onnxruntime
!pip install pyodbc
!pip install setuptools
!pip install skl2onnx
!pip install sqlalchemy

SET VALUES

In [40]:
server = 'iotedgevm-be9601db-be9601db.eastus.cloudapp.azure.com' # SQL Server IP address
username = 'sa' # SQL Server username
password = 'SuperSecretP@ssw0rd!' # SQL Server password
database = 'IoTEdgeDB'
data_table = 'OpcNodes'
models_table = 'Models'
features_table = 'Features'
target_table = 'Target'

master_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=master;UID=" + username + ";PWD=" + password + ";"
db_connection_string = "Driver={ODBC Driver 17 for SQL Server};Server=" + server + ";Database=" + database + ";UID=" + username + ";PWD=" + password + ";"

CREATE X TRAIN DATASET

In [6]:
import numpy as np
import onnxmltools
import onnxruntime as rt
import pandas as pd
import skl2onnx
import sklearn
import sklearn.datasets
import pyodbc

conn = pyodbc.connect(db_connection_string, autocommit=True)

query = 'SELECT ApplicationUri, DipData, SpikeData, RandomSignedInt32 FROM OpcNodes WHERE DipData IS NOT NULL AND SpikeData IS NOT NULL AND RandomSignedInt32 IS NOT NULL'
sql_query = pd.read_sql_query(query, conn)
x_train = pd.DataFrame(sql_query, columns=['DipData', 'SpikeData', 'RandomSignedInt32'])

print("\n*** Training dataset x\n")
print(x_train.head())


*** Training dataset x

          ApplicationUri    DipData  SpikeData  RandomSignedInt32
0  urn:OpcPlc:opcserver1  99.802673  99.802673       -164858984.0
1  urn:OpcPlc:opcserver1  99.802673  99.802673       -164858984.0
2  urn:OpcPlc:opcserver3 -90.482705 -90.482705       -153755509.0
3  urn:OpcPlc:opcserver3 -84.432793 -90.482705       -153755509.0
4  urn:OpcPlc:opcserver1  99.802673  99.802673       -164858984.0


CREATE Y TRAIN DATASET

In [22]:
target_column = 'MEDV'
y_train = pd.DataFrame()

y_train[target_column] = np.random.randint(100, size=(len(x_train)))

print("\n*** Training dataset y\n")
print(y_train.head())


*** Training dataset y

   MEDV
0    46
1    43
2    96
3    61
4    39


CREATE PIPELINE TO TRAIN THE LINEAR REGRESSION MODEL

In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

continuous_transformer = Pipeline(steps=[('scaler', RobustScaler())])

# All columns are numeric - normalize them
preprocessor = ColumnTransformer(
    transformers=[
        ('continuous', continuous_transformer, [i for i in range(len(x_train.columns))])])

model = Pipeline(
    steps=[
        ('preprocessor', preprocessor),
        ('regressor', LinearRegression())])

# Train the model
model.fit(x_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('continuous',
                                                  Pipeline(steps=[('scaler',
                                                                   RobustScaler())]),
                                                  [0, 1, 2])])),
                ('regressor', LinearRegression())])

CHECK MODEL ACCURACY

In [33]:
# Score the model
from sklearn.metrics import r2_score, mean_squared_error
y_pred = model.predict(x_train)
sklearn_r2_score = r2_score(y_train, y_pred)
sklearn_mse = mean_squared_error(y_train, y_pred)
print('*** Scikit-learn r2 score: {}'.format(sklearn_r2_score))
print('*** Scikit-learn MSE: {}'.format(sklearn_mse))

*** Scikit-learn r2 score: 0.0009219027813874847
*** Scikit-learn MSE: 823.7350351654205


DEFINE FUNCTION TO CONVERT MODEL TO ONNX

In [34]:
from skl2onnx.common.data_types import FloatTensorType, Int64TensorType, DoubleTensorType

def convert_dataframe_schema(df, drop=None, batch_axis=False):
    inputs = []
    nrows = None if batch_axis else 1
    for k, v in zip(df.columns, df.dtypes):
        if drop is not None and k in drop:
            continue
        if v == 'int64':
            t = Int64TensorType([nrows, 1])
        elif v == 'float32':
            t = FloatTensorType([nrows, 1])
        elif v == 'float64':
            t = DoubleTensorType([nrows, 1])
        else:
            raise Exception("Bad type")
        inputs.append((k, t))
    return inputs

SAVE MODEL IN ONNX FORMAT

In [48]:
# Convert the scikit model to onnx format
onnx_model = skl2onnx.convert_sklearn(model, 'OPC tags data', convert_dataframe_schema(x_train), final_types=[('variable1',FloatTensorType([1,1]))], target_opset=11)
# Save the onnx model locally
onnx_model_path = 'opcnodes.model.onnx'
onnxmltools.utils.save_model(onnx_model, onnx_model_path)

TEST ONNX MODEL

In [49]:
import onnxruntime as rt
sess = rt.InferenceSession(onnx_model_path)

y_pred = np.full(shape=(len(x_train)), fill_value=np.nan)

for i in range(len(x_train)):
    inputs = {}
    for j in range(len(x_train.columns)):
        inputs[x_train.columns[j]] = np.full(shape=(1,1), fill_value=x_train.iloc[i,j])

    sess_pred = sess.run(None, inputs)
    y_pred[i] = sess_pred[0][0][0]

onnx_r2_score = r2_score(y_train, y_pred)
onnx_mse = mean_squared_error(y_train, y_pred)

print()
print('*** Onnx r2 score: {}'.format(onnx_r2_score))
print('*** Onnx MSE: {}\n'.format(onnx_mse))
print('R2 Scores are equal' if sklearn_r2_score == onnx_r2_score else 'Difference in R2 scores: {}'.format(abs(sklearn_r2_score - onnx_r2_score)))
print('MSE are equal' if sklearn_mse == onnx_mse else 'Difference in MSE scores: {}'.format(abs(sklearn_mse - onnx_mse)))
print()


*** Onnx r2 score: 0.0009219022185058545
*** Onnx MSE: 823.7350356295136

Difference in R2 scores: 5.628816301950224e-10
Difference in MSE scores: 4.640930910682073e-07



<u>INSERT MODEL IN DATABASE</u>

In [57]:
import pyodbc

# Insert the ONNX model into the models table for each value of ApplicationUri
query = f'SELECT DISTINCT ApplicationUri FROM {data_table}'
cursor = conn.cursor()
cursor.execute(query)
application_uris = cursor.fetchall()

for app in application_uris:
    query = f"insert into {models_table} ([applicationUri], [description], [data]) values ('{app[0]}', 'Onnx Model', ?)"
    model_bits = onnx_model.SerializeToString()
    insert_params  = (pyodbc.Binary(model_bits))
    cursor.execute(query, insert_params)
    conn.commit()

ENSURE MODEL HAS BEEN STORED IN TABLE

In [58]:
conn = pyodbc.connect(db_connection_string, autocommit=True)
cursor = conn.cursor()
query = f'SELECT * FROM {models_table}'
cursor.execute(query)

for row in cursor.fetchall():
    print(row)


(3, b'\x08\x06\x12\x08skl2onnx\x1a\x051.9.0"\x07ai.onnx(\x002\x00:\xfc\x06\nE\n\x11RandomSignedInt32\x12\x16RandomSignedInt32_cast\x1a\x05Cast2"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n0\n\x07DipData\x12\x0cDipData_cast\x1a\x04Cast"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\n5\n\tSpikeData\x12\x0eSpikeData_cast\x1a\x05Cast1"\x04Cast*\t\n\x02to\x18\x0b\xa0\x01\x02:\x00\ne\n\x0cDipData_cast\n\x0eSpikeData_cast\n\x16RandomSignedInt32_cast\x12\x0emerged_columns\x1a\x06Concat"\x06Concat*\x0b\n\x04axis\x18\x01\xa0\x01\x02:\x00\n1\n\x0emerged_columns\n\tSu_Subcst\x12\x05Su_C0\x1a\x06Su_Sub"\x03Sub:\x00\n+\n\x05Su_C0\n\tDi_Divcst\x12\x08variable\x1a\x06Di_Div"\x03Div:\x00\n.\n\x08variable\n\x04coef\x12\nmultiplied\x1a\x06MatMul"\x06MatMul:\x00\n)\n\nmultiplied\n\tintercept\x12\x04resh\x1a\x03Add"\x03Add:\x00\n3\n\x04resh\n\x0cshape_tensor\x12\tvariable2\x1a\x07Reshape"\x07Reshape:\x00\n0\n\tvariable2\x12\tvariable1\x1a\x05Cast3"\x04Cast*\t\n\x02to\x18\x01\xa0\x01\x02:\x00\x12\rOPC tags 

CREATE FEATURES AND TARGET TABLES

In [93]:
import sqlalchemy
from sqlalchemy import create_engine
import urllib

conn = pyodbc.connect(db_connection_string)
cursor = conn.cursor()

features_table_name = 'features'

# Drop the table if it exists
query = f'drop table if exists {features_table_name}'
cursor.execute(query)
conn.commit()

# Create the features table
query = \
    f'create table {features_table_name} ( ' \
    f'    [CRIM] float, ' \
    f'    [ZN] float, ' \
    f'    [INDUS] float, ' \
    f'    [CHAS] float, ' \
    f'    [NOX] float, ' \
    f'    [RM] float, ' \
    f'    [AGE] float, ' \
    f'    [DIS] float, ' \
    f'    [RAD] float, ' \
    f'    [TAX] float, ' \
    f'    [PTRATIO] float, ' \
    f'    [B] float, ' \
    f'    [LSTAT] float, ' \
    f'    [id] int)'

cursor.execute(query)
conn.commit()

target_table_name = 'target'

query = f'drop table if exists {target_table_name}'
cursor.execute(query)
conn.commit()

# Create the target table
query = \
    f'create table {target_table_name} ( ' \
    f'    [MEDV] float, ' \
    f'    [id] int)'

cursor.execute(query)
conn.commit()

x_train['id'] = range(1, len(x_train)+1)
y_train['id'] = range(1, len(y_train)+1)

print(x_train.head())
print(y_train.head())

      CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  id  
0     15.3  396.90   4.98   1  
1     17.8  396.90   9.14   2  
2     17.8  392.83   4.03   3  
3     18.7  394.63   2.94   4  
4     18.7  396.90   5.33   5  
   MEDV  id
0  24.0   1
1  21.6   2
2  34.7   3
3  33.4   4
4  36.2   5


INSERT FEATURES & TARGET DATASETS INTO THE DATABASE

In [95]:
cursor = conn.cursor()

# for index, row in x_train.iterrows():
#     # print(index)
#     cursor.execute(f'INSERT INTO {features_table_name} (CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,id) VALUES ({row.CRIM},{row.ZN},{row.INDUS},{row.CHAS},{row.NOX},{row.RM},{row.AGE},{row.DIS},{row.RAD},{row.TAX},{row.PTRATIO},{row.B},{row.LSTAT},{row.id})')

# conn.commit()

for index, row in y_train.iterrows():
    # print(index)
    cursor.execute(f'INSERT INTO {target_table_name} (MEDV,id) VALUES ({row.MEDV},{row.id})')

conn.commit()
cursor.close()

In [ ]:
RUN PREDICT STATEMENT (SWITCH TO SQL KERNEL)

In [2]:
USE IoTEdgeDB

DECLARE @model VARBINARY(max) = (
    SELECT DATA
    FROM dbo.Models
    WHERE applicationUri = 'urn:OpcPlc:opcserver1'
    );

WITH predict_input
AS (
    SELECT TOP (1000) [id]
        , DipData
        , SpikeData
        , RandomSignedInt32
    FROM [dbo].[OpcNodes]
    )
SELECT predict_input.id
    , p.variable1 AS MEDV
FROM PREDICT(MODEL = @model, DATA = predict_input, RUNTIME=ONNX) WITH (variable1 FLOAT) AS p;


(410 rows affected)

Total execution time: 00:00:00.084

id,MEDV
1,48.7689094543457
2,48.7689094543457
3,48.606109619140625
4,49.06769943237305
5,49.04322052001953
6,49.04322052001953
7,49.666194915771484
8,48.90781021118164
9,48.97811508178711
10,50.00716018676758
